In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas import DataFrame
import psycopg2

In [2]:
# Store insurance csv into a dataframe
csv_file = "../Resources/insurance.csv"
insurance_data_df = pd.read_csv(csv_file)
insurance_data_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [32]:
# Value count to identify unique values - Gender count
count = insurance_data_df["sex"].value_counts()
count

male      676
female    662
Name: sex, dtype: int64

In [5]:
# Create a filtered dataframe from specific columns - multiples for insurance based on Gender and Charges
insurance_data_df = insurance_data_df[['sex', 'age', 'bmi', 'charges']].copy()
insurance_data_df.head()

,sex,age,bmi,charges
0,female,19,27.900,16884.92400
1,male,18,33.770,1725.55230
2,male,28,33.000,4449.46200
3,male,33,22.705,21984.47061
4,male,32,28.880,3866.85520


In [6]:
# Statistical data conversion for insurance data source
insurance_data_df.describe()

,age,bmi,charges
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,13270.422265
std,14.049960,6.098187,12110.011237
min,18.000000,15.960000,1121.873900
25%,27.000000,26.296250,4740.287150
50%,39.000000,30.400000,9382.033000
75%,51.000000,34.693750,16639.912515
max,64.000000,53.130000,63770.428010


In [4]:
# Store csv into a dataframe
csv_file = "../Resources/hospital.csv"
hospital_data_df = pd.read_csv(csv_file)
hospital_data_df.head()

,AGE,FEMALE,LOS,RACE,TOTCHG,APRDRG
0,17,1,2,1.0,2660,560
1,17,0,2,1.0,1689,753
2,17,1,7,1.0,20060,930
3,17,1,1,1.0,736,758
4,17,1,1,1.0,1194,754


In [7]:
# Display of the hospital dataset
hospital_data_df.head(10)

,AGE,FEMALE,LOS,RACE,TOTCHG,APRDRG
0,17,1,2,1.0,2660,560
1,17,0,2,1.0,1689,753
2,17,1,7,1.0,20060,930
3,17,1,1,1.0,736,758
4,17,1,1,1.0,1194,754
5,17,0,0,1.0,3305,347
6,17,1,4,1.0,2205,754
7,16,1,2,1.0,1167,754
8,16,1,1,1.0,532,753
9,17,1,2,1.0,1363,758


In [8]:
# Statistical view of the hospital dataset
hospital_data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
AGE,500.0,5.086000,6.949474,0.0,0.00,0.0,13.00,17.0
FEMALE,500.0,0.512000,0.500357,0.0,0.00,1.0,1.00,1.0
LOS,500.0,2.828000,3.363487,0.0,2.00,2.0,3.00,41.0
RACE,499.0,1.078156,0.514746,1.0,1.00,1.0,1.00,6.0
TOTCHG,500.0,2774.388000,3888.407367,532.0,1216.25,1536.5,2530.25,48388.0
APRDRG,500.0,616.360000,178.316044,21.0,640.00,640.0,751.00,952.0


In [9]:
# Create database connection
joining_string = "admin1:12345@localhost:5432/ETLProject"
engine = create_engine(f'postgresql://{joining_string}')

In [10]:
# Confirm Tables
engine.table_names()

['insurance_data', 'hospital_data']

In [11]:
insurance_data_df.to_sql(name="insurance_data", con=engine, if_exists='append', index=False)


In [12]:
hospital_data_df.to_sql(name="hospital_data", con=engine, if_exists='append', index=False)

In [13]:
pd.read_sql_query('SELECT * FROM insurance_data', con=engine).head(10)

,sex,age,bmi,charges
0,female,19,27.900,16884.92400
1,male,18,33.770,1725.55230
2,male,28,33.000,4449.46200
3,male,33,22.705,21984.47061
4,male,32,28.880,3866.85520
5,female,31,25.740,3756.62160
6,female,46,33.440,8240.58960
7,female,37,27.740,7281.50560
8,male,37,29.830,6406.41070
9,female,60,25.840,28923.13692


In [14]:
pd.read_sql_query('SELECT * FROM hospital_data', con=engine).head(10)

,AGE,FEMALE,LOS,RACE,TOTCHG,APRDRG
0,17,1,2,1.0,2660,560
1,17,0,2,1.0,1689,753
2,17,1,7,1.0,20060,930
3,17,1,1,1.0,736,758
4,17,1,1,1.0,1194,754
5,17,0,0,1.0,3305,347
6,17,1,4,1.0,2205,754
7,16,1,2,1.0,1167,754
8,16,1,1,1.0,532,753
9,17,1,2,1.0,1363,758


In [15]:
# Create bins in which to place values based upon age
bins = [-1,18, 24, 34, 44, 54, 65]
# Create labels for these bins
group_labels = ['0 to 17','18 to 24', '25 to 34', '35 to 44', '45 to 54', '55 to 64']

In [16]:
# Slice the data and place it into bins
pd.cut(hospital_data_df["AGE"], bins, labels=group_labels)

0      0 to 17
1      0 to 17
2      0 to 17
3      0 to 17
4      0 to 17
        ...   
495    0 to 17
496    0 to 17
497    0 to 17
498    0 to 17
499    0 to 17
Name: AGE, Length: 500, dtype: category
Categories (6, object): [0 to 17 < 18 to 24 < 25 to 34 < 35 to 44 < 45 to 54 < 55 to 64]

In [17]:
print(insurance_data_df['age'].mean,())
print(hospital_data_df['AGE'].mean())

<bound method Series.mean of 0       19
1       18
2       28
3       33
4       32
        ..
1333    50
1334    18
1335    18
1336    21
1337    61
Name: age, Length: 1338, dtype: int64> ()
5.086


In [18]:
print(insurance_data_df['charges'].mean,())
print(hospital_data_df['TOTCHG'].mean())

<bound method Series.mean of 0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64> ()
2774.388


In [19]:
insurance_data_df.groupby(by='age')['charges'].mean()

age
18     7086.217556
19     9747.909335
20    10159.697736
21     4730.464330
22    10012.932802
23    12419.820040
24    10648.015962
25     9838.365311
26     6133.825309
27    12184.701721
28     9069.187564
29    10430.158727
30    12719.110358
31    10196.980573
32     9220.300291
33    12351.532987
34    11613.528121
35    11307.182031
36    12204.476138
37    18019.911877
38     8102.733674
39    11778.242945
40    11772.251310
41     9653.745650
42    13061.038669
43    19267.278653
44    15859.396587
45    14830.199856
46    14342.590639
47    17653.999593
48    14632.500445
49    12696.006264
50    15663.003301
51    15682.255867
52    18256.269719
53    16020.930755
54    18758.546475
55    16164.545488
56    15025.515837
57    16447.185250
58    13878.928112
59    18895.869532
60    21979.418507
61    22024.457609
62    19163.856573
63    19884.998461
64    23275.530837
Name: charges, dtype: float64

In [20]:
hospital_data_df.groupby(by='LOS')['TOTCHG'].mean()

LOS
0      1606.200000
1      1907.721519
2      1707.986607
3      2537.367347
4      3415.526316
5      5372.500000
6      8370.500000
7     12307.272727
8      5014.000000
9     16520.000000
10     5615.000000
12    10969.500000
15     8631.000000
17    12042.000000
18    11167.000000
23    13112.000000
24    13040.000000
39    26356.000000
41    29188.000000
Name: TOTCHG, dtype: float64

In [21]:
# Descriptive statistics on the Hospital dataframe
hospital_data_df.groupby(by='AGE')['TOTCHG'].describe()

,count,mean,std,min,25%,50%,75%,max
AGE,,,,,,,,
0,307.0,2208.853420,2784.135079,550.0,1263.00,1483.0,2015.50,29188.0
1,10.0,3774.400000,2932.268148,1409.0,1666.00,2524.5,4751.75,9606.0
2,1.0,7298.000000,NaN,7298.0,7298.00,7298.0,7298.00,7298.0
3,3.0,10183.333333,3516.461337,8084.0,8153.50,8223.0,11233.00,14243.0
4,2.0,7996.000000,1745.139536,6762.0,7379.00,7996.0,8613.00,9230.0
5,2.0,9253.500000,1881.611145,7923.0,8588.25,9253.5,9918.75,10584.0
6,2.0,8964.000000,800.444876,8398.0,8681.00,8964.0,9247.00,9530.0
7,3.0,3362.333333,2742.908371,1132.0,1831.00,2530.0,4477.50,6425.0
8,2.0,2370.500000,1721.805012,1153.0,1761.75,2370.5,2979.25,3588.0


In [22]:
# Descriptive statistics on the Insurance dataframe
insurance_data_df.groupby(by='age')['charges'].describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
18,69.0,7086.217556,10198.459989,1121.87390,1633.044400,2198.189850,11272.331390,38792.68560
19,68.0,9747.909335,12469.537468,1241.56500,1639.563100,2135.891875,17148.980075,39722.74620
20,29.0,10159.697736,12049.624794,1391.52870,1967.022700,2459.720100,17085.267600,38344.56600
21,28.0,4730.464330,6168.059334,1515.34490,1985.288350,2254.423600,3205.349713,26018.95052
22,28.0,10012.932802,14653.363670,1664.99960,2155.351375,2641.155700,4078.134675,44501.39820
23,28.0,12419.820040,13421.332226,1815.87590,2432.780150,3594.538000,19145.024148,40904.19950
24,28.0,10648.015962,12203.650633,1969.61400,2462.839600,3045.137650,17508.424985,38126.24650
25,28.0,9838.365311,11551.289468,2137.65360,3160.320800,3750.148925,16333.214445,42112.23560
26,28.0,6133.825309,7765.729490,2302.30000,2921.320525,3388.882175,3983.464100,36085.21900


In [ ]:
# Data sources pulled from Kaggle. The two datasets used are 'Medical Costs Personal Datasets' & 'Hospital Costs'
# The datasets provide information from the insurance perspective and the hospital perspective, which vary based on a few key components.
# There are several variables associated with charges. The prevalent aspects effecting charges are 'Region', 'Gender', 'LOS(Length of stay)', 'Age', 'Level of Service'
# which is also dictated by the APRDRG (Coding formulary). 

In [ ]:
# The datasets were scrubbed to provide various scenarios of the information contained in the datasets (e.g. Insurance & Hospital) 
# Both datasets consisted of the age group from birth - 64; as we didn't include the Medicare aged population (those 65+) as the charges are 
# based on the Medicare Fee Schedule and highly regulated (Federal Government). 
# Thus, our final table used reflects the statistical data relevant to both age/charges in the Insurance and Hospital categories. 

In [ ]:
# ETL Project partners: Kruti Gandhi and Louis Starr.